In [ ]:
!pip install boto3

In [ ]:
import boto3
import pandas as pd
from json import dumps
import logging
import random

# Setup basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Provide your AWS credentials and region
aws_access_key_id = 'YOUR KEY ID'
aws_secret_access_key = 'YOUR ACCESS KEY'
region_name = 'us-east-2'
stream_name = 'stock_market_real_time_kinesis_data_stream'

# Load your data into a DataFrame
df = pd.read_csv('indexProcessed.csv')

# Create a Kinesis client
kinesis_client = boto3.client('kinesis', region_name=region_name,
                              aws_access_key_id=aws_access_key_id,
                              aws_secret_access_key=aws_secret_access_key)

# Infinite loop to continuously generate and publish events
while True:
    try:
        # Randomly sample one record from the DataFrame and convert it to a dictionary
        dict_stock = df.sample(1).to_dict(orient='records')[0]
        serialized_data = dumps(dict_stock)
        print(serialized_data)

        # Send the serialized data as a record to the Kinesis data stream
        response = kinesis_client.put_record(
            StreamName=stream_name,
            Data=serialized_data,
            PartitionKey=str(random.randint(0, 10000))  # Random partition key for load distribution
        )
        logging.info(f"Record sent successfully, SequenceNumber: {response['SequenceNumber']}")

    except Exception as e:
        logging.error(f"Failed to send record: {str(e)}")
